### RAG(Retriveal-Augmented-Generation): 검색 증강 생성
- 자연어 처리(NLP) 및 기계 학습 분야, 특히 챗복이나 질문-응답 시스템과 같은 고급 언어 모델을 구축하는 데 사용되는 기술

### NLP의 두 가지 주요 구성 요소인 정보 검색과 응답 생성을 결합
- 검색 부분은 관련 정보를 찾기 위해 대규모 데이터베이스나 문서 컬렉션을 검색하는 과정을 포함
- 생성 부분은 검색된 정보를 바탕으로 일관되고 맥락적으로 적절한 텍스트를 생성하는 과정

### 작동 방식
- 질문이나 프롬프트가 주어지면 모델은 먼저 질문에 대한 답변을 제공하는 데 유용한 정보를 포함할 수 있는 관련 문서나 텍스트를 검색
- 검색된 정보를 생성 모델에 공급하여 일관된 응답을 합성
> 질문 / 질문을 할 때 저장된 vector에서 단어를 탐색(유, 무 아님) -> 관련 문서를 prompt로 보낸다 -> model로 해당 prompt를 전달

#### 장점:
- 모델이 외부 지식을 활용할 수 있게 하여 보다 정확하고 상세하며 맥락적으로 관련된 답변을 제공
> 특정 지식이나 사실적 정보가 필요한 질문에 특히 유용

#### 응용 분야:
- 챗봇, 질문-응답 시스템 및 정확하고 맥락적으로 관련된 정보를 제공하는 것이 중요한 다른 AI 도구와 같은 다양한 응용 분야에 사용
- 다양한 주제와 데이터를 기반으로 이해하고 응답을 생성해야 하는 상황에서 유용

### data Retrieve 
- Source -> Load -> Transform -> Embed -> Storage -> Retrieve
- 소스에서 데이터 load 후, 데이터 분할하며 변환 -> 변환한 데이터를 임베드(텍스트에서 컴퓨터가 이해할 수 있는 숫자로 변환) -> 저장 / 검색

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader, PyPDFLoader
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    separators="\n",
    chunk_size=600,
    chunk_overlap=100,
)
# chunk_size=200, chunk_overlap=50
# chunk_overlap: 문장이나 문단을 분할할 때 앞 조각 끝 일부분을 가져오게 만든다. -> 중복이 생긴다.
# length_function=len: 얼마나 많은 글자가 있는지 세 준다.

# loader = TextLoader("./files/chapter_one.txt")
# loader = PyPDFLoader("./files/chapter_one.pdf")
# docs = loader.load()

loader = UnstructuredFileLoader("./files/chapter_one.docx")

# len(loader.load_and_split(text_splitter=splitter))
loader.load_and_split(text_splitter=splitter)



In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader, PyPDFLoader
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    separators="\n",
    chunk_size=600,
    chunk_overlap=100,
    
)
# length_function=len: 얼마나 많은 글자가 있는지 세 준다.
# token은 문자와 같은 의미가 아니다 -> 문자 두, 세 개를 한 개의 token으로 취급하기도 한다.

loader = UnstructuredFileLoader("./files/chapter_one.docx")

loader.load_and_split(text_splitter=splitter)

Vectors
    vecroization(벡터화) / 우리가 만든 문서마다 각각의 벡터를 만든다.
    word(단어)에 embed 작업을 한다.

Embed
    Embedding = 사람이 읽는 텍스트를 컴퓨터가 이해할 수 있는 숫자들로 변환하는 작업


In [ ]:
from langchain.embeddings import OpenAIEmbeddings

embedder = OpenAIEmbeddings()

# vector = embedder.embed_query("Hi") -> 단어 하나
vector = embedder.embed_documents([
    "hi",
    "how",
    "are",
    "you"
])
vector

In [29]:
# Vector Store
# 일종의 데이터베이스 

from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.storage import LocalFileStore

# CacheBackedEmbeddings, LocalFileStore : 파일 embedding 작업을 할 때 
# 캐시에 embeddings가 이미 존재하는지 확인, 없다면 vector store (Chroma.from_documents)를 호출 할 때 문서들과 함께 OpenAIEmbddings를 사용
# 처음엔 당연히 OpenAIEmbeddings를 사용 / 두 번째 호출부터 캐시에 저장되어 있는 embedding들을 가져온다.

cache_dir = LocalFileStore("./.cache/")

splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    separators="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = UnstructuredFileLoader("./files/chapter_one.docx")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir
)

vectorstore = Chroma.from_documents(docs, cached_embeddings)

In [28]:
results = vectorstore.similarity_search("where does winston live")
len(results)

4

In [ ]:
# LangSmith
# 우리의 체인이 무엇을 하고있는지 시각적으로 볼 수 있다.
# https://www.langchain.com/langsmith 로그인 ->  env에 등록
# chain을 실행할 때마다 작업을 project에서 볼 수 있다.

In [2]:
# off-the-shelf chain 사용 
# off-the-shelf chain은 사전 제작된 컴포넌트 조합으로, 편하게 사용하라고 만들어둔 체인

# 이후에 chain을 LangChain Expression Language로 만든다.
# 모든 설정을 한 눈에 보기 보기 쉽다. -> 어떤 값을 전달하면 Document Chain이 만들어지는데 우리의 dociment에 대해 즉시 답변할 것이다.

# 1. stuff(채우기)
# 내가 찾은 document들로 prompt를 stuff(채우기) 하는데 사용
# 질문 -> 질문을 사용해 document를 search -> 찾은 document들을 Prompt에 입력해서 model에게 전달 -> model은 입력된 질문과 documents를 토대로 우리에게 답변해준다.

from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import Chroma, FAISS
# FAISS가 성능이 Chroma 보다 좋다
from langchain.storage import LocalFileStore
from langchain.chains import RetrievalQA

llm = ChatOpenAI()

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = UnstructuredFileLoader("./files/chapter_one.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir
)

# vectorstore = Chroma.from_documents(docs, cached_embeddings)
vectorstore = FAISS.from_documents(docs, cached_embeddings)

chain = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type="map_rerank", #stuff, refine
    retriever= vectorstore.as_retriever(),
)
# retriver : document들을 database에서 검색할 수 있고, cloud나 vector store에서 찾아올 수도 있다.

# chain.run("Where does Winston live?")
chain.run("Describe Victory Mansions")

# LangSmith
# 1. 빅토리 멘션에 대해 묘사하라는 질문을 Retriever에게 전달
# 2. Refine documents chain의 작업 시작 -> 1) LLM Chain을 실행

/Users/jangtaehun/Desktop/study/nomad/Fullstack-GPT/env/lib/python3.11/site-packages/langchain/chains/llm.py:349: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


"Victory Mansions is a building with a gritty, smelly hallway that smells of boiled cabbage and old rag mats. A large colored poster with a man's face adorns one wall, depicting a man with a black mustache and ruggedly handsome features. The building has a faulty elevator and is seven flights up, with a poster of Big Brother watching on each landing. Inside the flat, there is a telescreen that cannot be completely shut off, constantly broadcasting figures related to pig-iron production. The building has a window from which a small, frail figure in blue overalls looks out, with fair hair and a sanguine face roughened by harsh conditions."

In [ ]:
# 1. stuff(채우기)
# 내가 찾은 document들로 prompt를 stuff(채우기) 하는데 사용
# 질문 -> 질문을 사용해 document를 search -> 찾은 document들을 Prompt에 입력해서 model에게 전달 -> model은 입력된 질문과 documents를 토대로 우리에게 답변해준다.

# 2. Refine(정제, 가다듬기)
# 질문을 통해 관련된 documents를 얻는다. 그리고 각각의 document를 읽으면서 질문에 대한 답변 생성 시도
# 반복하며 만나는 모든 document를 통해 question을 개선시킨다.
# 처음에는 질이 낮은 답변 -> 첫 번째 document를 읽고 그것을 기반으로 답변을 업데이트...
# 비싸다(개별적으로 하나의 답변을 생성해야 하므로 document가 10개면 10번 질문)


# 3. map reduce
# document들을 입력받아서 개별적으로 요약 작업을 수행 -> 각 document를 순회하면서 개별 답변을 찾아낸 후 탐색이 끝나면 일종의 중간 답변들을 기반으로 최종 응답을 반환
# quert를 입력하면 documents들을 얻어서 각각에 대한 요약 작업을 한다. 그리고 각각의 요약본을 LLM에게 전달해준다.


# 4. map re-rank
# 각 document를 순회하면서 작업을 시작하는데 단순히 document의 답변을 추출하는 대신 각 document에 기반해서 질문에 대답하고 답변에 점수를 매긴다.
# 질문을 하면 관련된 document들을 받는다. -> 각 document를 통해 답변을 생성하고 각 답변에 점수를 매긴다.
# 최종적으로 가능 높은 점수를 획득한 답변과 그 점수를 함께 반환한다.

- fileloder -> UnstructuredFileLoader -> split: 긴 text를 다루기 위해 / 긴 text를 작은 document들로 나누기 위해
- 거대한 단일 document보다는 작은 여러 개를 LLM에게 전달할 때 검색 성능이 좋다.
- - 작게 분할하면, 작업이 쉬워지고 응답도 빨라지며 LLM 사용 비용이 줄어든다.

- 작업을 요약하면, document를 적재(load)하고 분할(split) 한다

- Embedding은 text에 의미별로 적절한 점수를 부여해서 vector 형식으로 표현한 것
- OpenAIEmbeddings을 사용했고 CacheBackedEmbeddings을 사용해 만들어진 Embedding을 cache(저장) -> 비용 절감

``` python
cache_dir = LocalFileStore("./.cache/")
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)
# 임베딩 모델을 전달하고 데이터가 저장될 폴더 지정
```

- Vector store를 호출(call) -> Chroma, FAISS 사용
``` python
vectorstore = FAISS.from_documents(docs, cached_embeddings)
# document와 embedding, from_documents 메서드를 호출
```
- 이 메서드는 document 별로 embedding 작업 후 결과를 저장한 vector store를 반환하는데 이를 이용해 document 검색도 하고 연관성이 높은 document들을 찾기도 한다.

- RetrievalQA이라는 chain 사용 -> (LLM, chain의 종류, retriever)
- - LangChain이 제공하는 class 또는 interface의 일종으로 document를 검색해 찾아오는(retrieve) 기능을 가지고 있다. / vector store를 retriever로 전환할 수도 있다

In [ ]:
# LCEL을 사용해 stuff chain 구현
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import Chroma, FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough

llm = ChatOpenAI()

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = UnstructuredFileLoader("./files/chapter_one.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir
)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriever = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 조수입니다. 주어진 {context}만을 이용해 질문에 대답하세요. 답을 모르면 모른다고 하세요. 모르는 정보를 지어내지마세요."),
    ("human", "{quesition}")
])

chain = {"context":retriever, "quesition": RunnablePassthrough()} | prompt | llm

chain.invoke("Describe Victory Mansions")
# 실행되면 quesition이 retriever에게 전달 -> document들의 list를 반환 -> 그 document들은 context값으로 입력되어야 한다.
# quesition도 prompt template의 quesition으로 입력되어야 한다.

# RunnablePassthrough: 입력값 을 전달할 수 있게 해준다.